In [ ]:
%load_ext jupyter_black

In [ ]:
import datetime
from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz
import threading
from time import sleep

from utils import calculate_pnl, plot_line, plot_quantiles

In [ ]:
DATA_DIR = "../trades/"
LOG_DIR = "../logs/"
PROCESSED_DIR = "./trades/"
NUM_RECENT_TRADES = 10

DATA_FILE = "wif-usdc"
START_DATETIME = ""
END_DATETIME = ""
PRICE_PER_TICK = 0.0001
UNITS_PER_LOT = 0.01

In [ ]:
def num_decimals(indicator):
    indicator_str = str(indicator)
    if "." in indicator_str:
        decimal_places = len(indicator_str.split(".")[1])
    else:
        decimal_places = 0

    return decimal_places

In [ ]:
TOKEN_ENTRY_LABEL_WIDTH = 100
TOKEN_ENTRY_WIDTH = 225

DATE_ENTRY_LABEL_WIDTH = 100
DATE_ENTRY_WIDTH = 400

CHART_WIDTH = 568
CHART_HEIGHT = 280
CHART_PADDING = 5
CHART_MARGIN = 1

filename_wid = widgets.Text(
    value=DATA_FILE,
    description="Filename:",
    style={"description_width": f"{TOKEN_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{TOKEN_ENTRY_WIDTH}px"),
)
price_per_tick_wid = widgets.FloatText(
    value=PRICE_PER_TICK,
    step=PRICE_PER_TICK,
    description="Price per Tick:",
    style={"description_width": f"{TOKEN_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{TOKEN_ENTRY_WIDTH}px"),
)
units_per_lot_wid = widgets.FloatText(
    value=UNITS_PER_LOT,
    step=UNITS_PER_LOT,
    description="Units per Lot:",
    style={"description_width": f"{TOKEN_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{TOKEN_ENTRY_WIDTH}px"),
)

start_time_wid = widgets.DatetimePicker(
    description="Start:",
    style={"description_width": f"{DATE_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{DATE_ENTRY_WIDTH}px"),
)
end_time_wid = widgets.DatetimePicker(
    description="End:",
    style={"description_width": f"{DATE_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{DATE_ENTRY_WIDTH}px"),
)

summary_out_wid = widgets.Output(
    layout={
        "border": "1px solid black",
        "padding": f"{CHART_PADDING}px",
        "margin": f"{CHART_MARGIN}px",
    }
)
pnl_out_wid = widgets.Output(
    layout={
        "border": "1px solid black",
        "padding": f"{CHART_PADDING}px",
        "margin": f"{CHART_MARGIN}px",
    }
)
inventory_out_wid = widgets.Output(
    layout={
        "border": "1px solid black",
        "padding": f"{CHART_PADDING}px",
        "margin": f"{CHART_MARGIN}px",
    }
)
size_out_wid = widgets.Output(
    layout={
        "border": "1px solid black",
        "padding": f"{CHART_PADDING}px",
        "margin": f"{CHART_MARGIN}px",
    }
)
timestamp_delta_out_wid = widgets.Output(
    layout={
        "border": "1px solid black",
        "padding": f"{CHART_PADDING}px",
        "margin": f"{CHART_MARGIN}px",
    }
)
timestamp_delay_out_wid = widgets.Output(
    layout={
        "border": "1px solid black",
        "padding": f"{CHART_PADDING}px",
        "margin": f"{CHART_MARGIN}px",
    }
)
log_out_wid = widgets.Output()
trades_out_wid = widgets.Output()

In [ ]:
spacer = widgets.Box(layout=widgets.Layout(height="20px"))

time_range_vbox = widgets.VBox([filename_wid, start_time_wid, end_time_wid])
instrument_vbox = widgets.VBox(
    [price_per_tick_wid, units_per_lot_wid],
)

pnl_inventory_vbox = widgets.VBox([pnl_out_wid, inventory_out_wid, size_out_wid])
timestamp_vbox = widgets.VBox(
    [summary_out_wid, timestamp_delta_out_wid, timestamp_delay_out_wid]
)

parameters_hbox = widgets.HBox([time_range_vbox, instrument_vbox])
charts_hbox = widgets.HBox([pnl_inventory_vbox, timestamp_vbox])

In [ ]:
trades = None


def interpolate_red_green(value, min_val, max_val):
    if value <= min_val:
        return "red"
    elif value >= max_val:
        return "green"
    else:
        ratio = (value - min_val) / (max_val - min_val)
        red = int(255 * (1 - ratio))
        green = int(255 * ratio)
        return f"rgb({red},{green},0)"


def show_summary():
    duration = trades["timestamp"].max() - trades["timestamp"].min()
    hours = duration.components.hours
    minutes = duration.components.minutes

    trade_volume = trades["size"] * trades["price"]
    quote_volume = trade_volume.sum()
    base_volume = trades["size"].sum()

    pnl_series = calculate_pnl(trades)

    fig = make_subplots(
        rows=2,
        cols=2,
        specs=[
            [{"type": "indicator"}, {"type": "indicator"}],
            [{"type": "indicator"}, {"type": "indicator"}],
        ],
    )

    fig.add_trace(
        go.Indicator(
            mode="number",
            value=len(trades),
            number={"font": {"size": 50}},
            title={"text": "Fills", "font": {"size": 15}},
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Indicator(
            mode="number",
            value=round((pnl_series.iloc[-1] / quote_volume) * 10000, 2),
            number={
                "font": {
                    "size": 50,
                    "color": interpolate_red_green(
                        (pnl_series.iloc[-1] / quote_volume) * 10000, -10, 10
                    ),
                }
            },
            title={"text": "PnL/Volume (bps)", "font": {"size": 15}},
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Indicator(
            mode="number",
            value=round(base_volume, num_decimals(units_per_lot_wid.value)),
            number={
                "font": {"size": 50},
                "valueformat": f".{num_decimals(units_per_lot_wid.value)}f",
            },
            title={"text": "Base Volume", "font": {"size": 15}},
        ),
        row=1,
        col=2,
    )

    fig.add_trace(
        go.Indicator(
            mode="number",
            value=round(quote_volume, 2),
            number={"font": {"size": 50}, "valueformat": ".2f"},
            title={"text": "Quote Volume", "font": {"size": 15}},
        ),
        row=2,
        col=2,
    )

    fig.update_layout(
        width=CHART_WIDTH, height=CHART_HEIGHT, margin=dict(l=0, r=0, t=20, b=0)
    )

    summary_out_wid.clear_output()
    with summary_out_wid:
        fig.show()


def update_trades():
    global trades

    try:
        data_path = f"{DATA_DIR}{filename_wid.value}.csv"
        trades = pd.read_csv(data_path)

        trades["timestamp"] = pd.to_datetime(trades["timestamp"], unit="s")
        trades["timestamp"] = (
            trades["timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
        )

        trades["local_timestamp"] = pd.to_datetime(trades["local_timestamp"], unit="ms")
        trades["local_timestamp"] = (
            trades["local_timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
        )

        trades["timestamp_delay"] = trades["local_timestamp"] - trades["timestamp"]
        trades["timestamp_delay"] = trades["timestamp_delay"].dt.total_seconds()

        trades = trades.sort_values(by="timestamp")

        trades["price"] = trades["price"] * price_per_tick_wid.value
        trades["size"] = trades["size"] * units_per_lot_wid.value

        trades["inventory_change"] = trades.apply(
            lambda row: row["size"] if row["side"] == "buy" else -row["size"], axis=1
        )
        trades["inventory"] = trades["inventory_change"].cumsum()

    except Exception as e:
        raise e

    trades_out_wid.clear_output()
    with trades_out_wid:
        display(
            HTML(
                trades[
                    [
                        "timestamp",
                        "timestamp_delay",
                        "slot",
                        "side",
                        "price",
                        "size",
                        "inventory",
                        "signature",
                    ]
                ]
                .tail(NUM_RECENT_TRADES)
                .sort_values(by="timestamp", ascending=False)
                .to_html(index=False)
            )
        )


def render(*args):
    try:
        update_trades()
    except Exception as e:
        log_out_wid.clear_output()
        with log_out_wid:
            print(f"Failed to fetch new trades: {e}")
        return

    pnl_series = calculate_pnl(trades)
    flat_pnl_series = pd.Series(0, index=pnl_series.index)
    inventory_series = trades.set_index("timestamp")["inventory"]
    flat_inventory_series = pd.Series(0, index=inventory_series.index)
    time_diffs = trades["timestamp"].diff().dt.total_seconds()
    time_diffs = time_diffs.dropna()

    show_summary()

    pnl_out_wid.clear_output()
    with pnl_out_wid:
        plot_line(
            trades["timestamp"],
            [pnl_series, flat_pnl_series],
            ["PnL", "Break Even"],
            "Time",
            "PnL",
            show_legend=False,
            width=CHART_WIDTH,
            height=CHART_HEIGHT,
        )

    inventory_out_wid.clear_output()
    with inventory_out_wid:
        plot_line(
            trades["timestamp"],
            [inventory_series, flat_inventory_series],
            ["Inventory", "Flat"],
            "Time",
            "Inventory",
            show_legend=False,
            width=CHART_WIDTH,
            height=CHART_HEIGHT,
        )

    size_out_wid.clear_output()
    with size_out_wid:
        plot_quantiles(
            trades["size"],
            round_decimals=num_decimals(units_per_lot_wid.value),
            bins=50,
            name="Size",
            width=CHART_WIDTH,
            height=CHART_HEIGHT,
        )

    timestamp_delta_out_wid.clear_output()
    with timestamp_delta_out_wid:
        plot_quantiles(
            time_diffs,
            round_decimals=2,
            bins=50,
            name="Time between Trades (s)",
            width=CHART_WIDTH,
            height=CHART_HEIGHT,
        )

    timestamp_delay_out_wid.clear_output()
    with timestamp_delay_out_wid:
        plot_quantiles(
            trades["timestamp_delay"],
            round_decimals=2,
            bins=50,
            name="Timestamp Delay (s)",
            width=CHART_WIDTH,
            height=CHART_HEIGHT,
        )


for widget in [
    filename_wid,
    price_per_tick_wid,
    units_per_lot_wid,
    start_time_wid,
    end_time_wid,
]:
    widget.observe(render, "value")


# def periodic_update():
#     while True:
#         sleep(5)
#         render()


# thread = threading.Thread(target=periodic_update, daemon=True)
# thread.start()

In [ ]:
display(parameters_hbox)
display(log_out_wid)
display(spacer)
display(charts_hbox)
display(spacer)
display(trades_out_wid)

render()

In [ ]:
# for some reason the voila view wont display the first two plotly figures
dummy_series = pd.Series(data=[1], index=[1])
plot_quantiles(dummy_series, bins=2, name="")
plot_quantiles(dummy_series, bins=2, name="")